In [ ]:
#@title 必要なライブラリのインストール

!pip install -q ultralytics
!git clone https://github.com/edihbrandon/RictyDiminished.git

import colorsys
import os
import random
import re

import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageFont
from pydantic import BaseModel
from tqdm.notebook import tqdm
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.4/910.4 kB 19.3 MB/s eta 0:00:00
Cloning into 'RictyDiminished'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 24 (from 1)
Receiving objects: 100% (25/25), 7.07 MiB | 9.41 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.



ライブラリのバージョンの確認方法
```
# OpenCVのバージョン
import cv2
print(f"OpenCV (cv2) バージョン: {cv2.__version__}")

# NumPyのバージョン
import numpy as np
print(f"NumPy バージョン: {np.__version__}")

# Matplotlibのバージョン
import matplotlib
print(f"Matplotlib バージョン: {matplotlib.__version__}")

# Pillowのバージョン
import PIL
print(f"Pillow バージョン: {PIL.__version__}")

# Pydanticのバージョン
import pydantic
print(f"Pydantic バージョン: {pydantic.__version__}")

# tqdmのバージョン
import tqdm
print(f"tqdm バージョン: {tqdm.__version__}")

# Ultralyticsのバージョン
import ultralytics
print(f"Ultralytics バージョン: {ultralytics.__version__}")
```

```
OpenCV (cv2) バージョン: 4.10.0
NumPy バージョン: 1.26.4
Matplotlib バージョン: 3.10.0
Pillow バージョン: 11.1.0
Pydantic バージョン: 2.10.4
tqdm バージョン: 4.67.1
Ultralytics バージョン: 8.3.59
```

In [ ]:
# @title Google Driveのマウント

from google.colab import drive

drive.mount("/content/drive")


Mounted at /content/drive


**動画のダウンロード**

下記のURLから動画ファイルをダウンロードしてください。サイズは1280×720にしてください。ダウンロード後、Google Colabにアップロードしてください。



**[動画のURL](https://pixabay.com/ja/videos/%E4%BA%BA-%E5%95%86%E6%A5%AD-%E5%BA%97-%E5%BF%99%E3%81%97%E3%81%84-%E3%83%A2%E3%83%BC%E3%83%AB-6387/)**


> <a href="https://pixabay.com/ja//?utm_source=link-attribution&utm_medium=referral&utm_campaign=video&utm_content=6387">Pixabay</a>が提供する<a href="https://pixabay.com/ja/users/coverr-free-footage-1281706/?utm_source=link-attribution&utm_medium=referral&utm_campaign=video&utm_content=6387">Coverr-Free-Footage</a>の動画



In [ ]:
# @title 動画のディレクトリ・ファイル名の指定

video_dir = "/content/drive/MyDrive/Pixabay"  # @param {type:"string"}
video_filename = "6387-191695740_medium.mp4"  # @param {type:"string"}


In [ ]:
# @title Configオブジェクト（設定値を格納するオブジェクト）


class Config(BaseModel):
    video_dir: str
    video_filename: str
    roi_points: list[tuple[int, int]]

    @property
    def video_src_path(self):
        return os.path.join(self.video_dir, self.video_filename)


In [ ]:
# @title BBoxオブジェクト

PATH_FONT = "/content/RictyDiminished/RictyDiminishedDiscord-Regular.ttf"
FONT_CACHE: dict[int, ImageFont.FreeTypeFont] = {}
CHAR_CACHE: dict[str, dict[int, tuple[bool, tuple[int, int, int, int], np.ndarray]]] = (
    {}
)
RED = (0, 0, 255)
DEFAULT_PADDING_TOP = 20
TEXT_OFFSET = 5


class BBox(BaseModel):
    left: float
    top: float
    right: float
    bottom: float
    score: float
    label: int

    @property
    def bottom_center(self) -> tuple[float, float]:
        return ((self.left + self.right) / 2, self.bottom)

    def draw(
        self,
        img: np.ndarray,
        classnames: dict[int, str],
        color_mapping: dict[str, tuple[int, int, int]],
        alert: bool,
        another_text_list: list[tuple[str, tuple[int, int, int]]],
    ):
        color = RED if alert else color_mapping.get(classnames[self.label], RED)

        # Draw bounding box
        cv2.rectangle(
            img,
            pt1=(int(self.left), int(self.top)),
            pt2=(int(self.right), int(self.bottom)),
            color=color,
            thickness=2,
        )

        # Draw label text
        self._put_text(img, classnames.get(self.label, "not found"), color, 0)

        # Draw additional texts
        padding_top = DEFAULT_PADDING_TOP
        for text, text_color in another_text_list:
            padding_top += self._put_text(img, text, text_color, padding_top)

    def _put_text(
        self, img: np.ndarray, text: str, color: tuple[int, int, int], padding_top: int
    ) -> int:
        text = re.sub(r"[\t\n\r]", "", text)
        h, w, *_ = img.shape
        d = img.ndim
        offset = TEXT_OFFSET
        max_height = 0

        for char in text:
            jp, bbox, mask = self._get_char_mask(char, size=20)
            li, ti, ri, bi = (
                int(self.left) + bbox[0] + offset,
                int(self.top) + bbox[1] + padding_top,
                int(self.left) + bbox[2] + offset,
                int(self.top) + bbox[3] + padding_top,
            )

            offset += bbox[2] - bbox[0]
            max_height = max(max_height, bbox[3] - bbox[1])

            # Adjust clipping
            lm, tm, rm, bm = 0, 0, ri - li, bi - ti
            li, lm = max(0, li), lm - min(0, li)
            ti, tm = max(0, ti), tm - min(0, ti)
            ri, rm = min(w, ri), rm - max(0, ri - w)
            bi, bm = min(h, bi), bm - max(0, bi - h)

            if (ri - li) <= 0 or (bi - ti) <= 0:
                continue

            # Apply mask to image
            if d == 3:
                img[ti:bi, li:ri] = (
                    img[ti:bi, li:ri] * (1 - mask[tm:bm, lm:rm, np.newaxis])
                    + color * mask[tm:bm, lm:rm, np.newaxis]
                )
            else:
                img[ti:bi, li:ri] = (
                    img[ti:bi, li:ri] * (1 - mask[tm:bm, lm:rm])
                    + color * mask[tm:bm, lm:rm]
                )

        return max_height

    @staticmethod
    def _get_char_mask(char: str, size: int):
        def _get_font(size: int) -> ImageFont.FreeTypeFont:
            if size not in FONT_CACHE:
                FONT_CACHE[size] = ImageFont.truetype(PATH_FONT, size)
            return FONT_CACHE[size]

        if char not in CHAR_CACHE:
            CHAR_CACHE[char] = {}

        if size not in CHAR_CACHE[char]:
            font = _get_font(size)
            bbox = font.getbbox(char)
            mask = (
                np.asarray(font.getmask(char, "L"), dtype=np.float32).reshape(
                    bbox[3] - bbox[1], bbox[2] - bbox[0]
                )
                / 255
            )
            CHAR_CACHE[char][size] = (not char.isascii(), bbox, mask)

        return CHAR_CACHE[char][size]


In [ ]:
# @title Detectionオブジェクト（結果を格納するオブジェクト）


class Detection:
    def __init__(self, bbox: BBox):
        self.bbox = bbox
        self.alert = False
        self.another_text_list: list[tuple[str, tuple[int, int, int]]] = []

    def draw(
        self,
        img: np.ndarray,
        classnames: dict[int, str],
        color_mapping: dict[str, tuple[int, int, int]],
    ):
        self.bbox.draw(
            img, classnames, color_mapping, self.alert, self.another_text_list
        )

    @classmethod
    def postprocess(cls, result) -> "Detection":
        return cls(cls._create_bbox(result))

    @staticmethod
    def _create_bbox(result) -> BBox:
        xyxy = result.boxes.xyxy.cpu().numpy()[0]
        return BBox(
            left=int(xyxy[0]),
            top=int(xyxy[1]),
            right=int(xyxy[2]),
            bottom=int(xyxy[3]),
            score=float(result.boxes.conf.cpu().numpy()[0]),
            label=int(result.boxes.cls.cpu().numpy()[0]),
        )


In [ ]:
# @title Driftingオブジェクト（フレームごとの結果を保存するオブジェクト）


class Drifting:
    def __init__(self, img, count, detections=[]):
        self.img: np.ndarray = img
        self.result_img: np.ndarray = img.copy()
        self.count: int = count
        self.detections: list[Detection] = detections


In [ ]:
# @title 動画読み込みエレメント


class VideoSrc:
    def __init__(self, config: Config):
        self.frame_num = 0
        self.video_src_path = config.video_src_path
        self.cap = cv2.VideoCapture(self.video_src_path)
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = round(self.cap.get(cv2.CAP_PROP_FPS))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    def read(self) -> Drifting | None:
        _, img = self.cap.read()

        if _ is False:
            return None

        drifting = Drifting(img, self.frame_num, [])
        print(f"\rcount: {self.frame_num} / {self.frame_count}", end="")
        self.frame_num += 1
        return drifting

    def reload(self):
        self.cap = cv2.VideoCapture(self.video_src_path)
        self.frame_num = 0

    def release(self):
        self.frame_num = 0
        self.cap.release()


In [ ]:
# @title 動画書き込みエレメント


class VideoSink:
    def __init__(self, src: VideoSrc):
        self.output_path = self.generate_output_path(src.video_src_path)
        fmt = cv2.VideoWriter_fourcc("m", "p", "4", "v")
        frame_rate = src.fps
        size = (src.width, src.height)
        self.writer = cv2.VideoWriter(self.output_path, fmt, frame_rate, size)

    def write(self, drifting: Drifting) -> Drifting:
        self.writer.write(drifting.result_img)
        return drifting

    def release(self):
        self.writer.release()

    @staticmethod
    def generate_output_path(file_path: str) -> str:
        file_name_without_ext, _ = os.path.splitext(file_path)
        new_file_path = f"{file_name_without_ext}_out.mp4"
        return new_file_path


In [ ]:
# @title 物体検出エレメント


class ObjectDetector:
    def __init__(self, conf=0.25):
        self.model = YOLO("yolov8m.pt")
        self.classnames = self.model.names
        self.confidence_threshold = conf

    def predict(self, drifting: Drifting) -> Drifting:
        results = self.model.predict(drifting.img, conf=self.confidence_threshold)
        drifting.detections += [Detection.postprocess(result) for result in results[0]]
        return drifting


In [ ]:
# @title 結果の描画エレメント


class DetectionRenderer:
    """物体検出の結果を画像上に可視化するためのクラス。

    各検出クラスに対して一意な色を割り当て、検出結果を視覚化します。
    人物クラスは緑色で、その他のクラスは黄金比を用いて自動的に色を生成します。
    """

    def __init__(self, classnames):
        """DetectionRendererを初期化します。

        Args:
            classnames: 検出対象のクラス名を含む辞書
        """
        # 黄金比を定数として保持
        self.GOLDEN_RATIO = 0.618033988749895
        self.classnames = classnames
        # 各クラスに対して色をマッピング
        self.color_mapping = {
            element: (0, 255, 0) if element == "person" else self._get_color(idx)
            for idx, element in enumerate(self.classnames.values())
        }

    def draw(self, drifting: Drifting) -> Drifting:
        """検出結果を画像上に描画します。

        Args:
            drifting: 検出結果と元画像を含むDriftingオブジェクト

        Returns:
            描画結果を含むDriftingオブジェクト
        """
        for det in drifting.detections:
            det.draw(drifting.result_img, self.classnames, self.color_mapping)
        return drifting

    def _get_color(
        self, idx: int, s: float = 0.8, vmin: float = 0.7
    ) -> tuple[int, int, int]:
        """インデックスから一意なBGR色を生成します。

        黄金比を使用して、視覚的に区別しやすい色の分布を生成します。
        HSV色空間で色を生成し、OpenCV用のBGR形式に変換します。

        Args:
            idx: クラスのインデックス
            s: 彩度(Saturation)の値。デフォルトは0.8
            vmin: 最小明度(Value)の値。デフォルトは0.7

        Returns:
            tuple[int, int, int]: BGRカラー値(0-255の範囲)
        """
        # 色相(Hue)の計算 - 黄金比を使用
        h = np.fmod(idx * self.GOLDEN_RATIO, 1.0)

        # 明度(Value)の計算
        v = 1.0 - np.fmod(idx * self.GOLDEN_RATIO, 1.0 - vmin)

        # HSVからRGBに変換
        r, g, b = colorsys.hsv_to_rgb(h, s, v)

        # BGRの順で返す（OpenCV形式）
        return (int(255 * b), int(255 * g), int(255 * r))


In [ ]:
# @title 侵入判定エレメント


def is_in_image_shape(img: np.ndarray, point: tuple) -> bool:
    """
    指定された点が画像の範囲内にあるかを判定する関数

    Args:
        img: 対象の画像（NumPy配列）
        point: 判定する点の座標 (x, y)

    Returns:
        bool: 点が画像範囲内にある場合True、そうでない場合False

    Note:
        img.shape[1]は画像の幅（x方向の大きさ）
        img.shape[0]は画像の高さ（y方向の大きさ）
    """
    return (0 <= point[0] < img.shape[1]) and (0 <= point[1] < img.shape[0])


class RoiWatcher:
    """
    関心領域（Region of Interest, ROI）内の物体検出を監視するクラス
    指定された多角形領域内に物体が侵入したかを判定する
    """

    def __init__(self, video_src, config: Config):
        """
        初期化メソッド

        Args:
            video_src: 動画ソースオブジェクト（高さと幅の情報を含む）
            points: ROIを定義する頂点座標のリスト
        """
        # ROIのマスク画像を初期化（0で初期化）
        self.roi_map = np.zeros((video_src.height, video_src.width), np.uint8)

        # ROI領域を示すピクセル値を定義
        self.PIXEL_VALUE_ROI = 255

        # 凸多角形としてROI領域を描画
        self.roi_map = cv2.fillConvexPoly(
            self.roi_map,  # 描画対象の画像
            points=np.array(config.roi_points),  # 多角形の頂点座標
            color=self.PIXEL_VALUE_ROI,  # 塗りつぶす色
        )

        # ROIの頂点座標を保存
        self.points = config.roi_points

    def detect(self, drifting):
        """
        検出された物体がROI内に侵入しているかを判定するメソッド

        Args:
            drifting: 検出結果と画像を含むDriftingオブジェクト

        Returns:
            Drifting: アラート情報を追加したDriftingオブジェクト
        """
        RED = (0, 0, 255)  # BGR形式での赤色

        # 各検出物体について処理
        for det in drifting.detections:
            # 物体の底部中心点が画像内かつROI内にあるか判定
            if (
                is_in_image_shape(drifting.img, det.bbox.bottom_center)
                and self.roi_map[int(det.bbox.bottom_center[1])][
                    int(det.bbox.bottom_center[0])
                ]
                == self.PIXEL_VALUE_ROI
            ):
                # ROI内に侵入している場合
                det.alert = True
                det.another_text_list.append(("侵入しています", RED))
            else:
                # ROI外の場合
                det.alert = False

        # ROIの可視化（頂点と辺の描画）
        for idx, pt in enumerate(self.points):
            # 頂点を赤い円で描画
            cv2.circle(
                drifting.result_img,  # 描画対象の画像
                pt,  # 円の中心座標
                10,  # 円の半径
                (0, 0, 255),  # 色（赤）
                -1,  # 塗りつぶし
            )

            # 頂点間を線で接続
            cv2.line(
                drifting.result_img,  # 描画対象の画像
                self.points[idx - 1],  # 開始点（前の頂点）
                self.points[idx],  # 終了点（現在の頂点）
                (0, 0, 255),  # 色（赤）
                thickness=1,  # 線の太さ
                lineType=cv2.LINE_4,  # 線の種類
            )

        return drifting


In [ ]:
# @title 設定の初期化

roi_points = [(536, 213), (513, 424), (838, 417), (789, 211)]  # 侵入禁止エリア

config = Config(
    video_dir=video_dir,  # 動画ファイルのディレクトリ
    video_filename=video_filename,  # 動画ファイル名
    roi_points=roi_points,
)


In [ ]:
# @title 処理パイプラインの構築

# 動画の入出力を準備
video_src = VideoSrc(config)  # 入力動画の読み込み
video_sink = VideoSink(video_src)  # 出力動画の設定

# 解析モジュールの準備
object_detector = ObjectDetector(conf=0.3)
detection_renderer = DetectionRenderer(object_detector.classnames)
roi_watcher = RoiWatcher(video_src, config)


100%|██████████| 49.7M/49.7M [00:00<00:00, 308MB/s]


In [ ]:
# @title 侵入検知

for _ in tqdm(range(video_src.frame_count)):
    # 1. フレームの読み込み
    drifting = video_src.read()
    if drifting is None:  # 動画終了のチェック
        break

    # 2. 物体検出の実行
    drifting = object_detector.predict(drifting)

    # 3. 侵入判定
    drifting = roi_watcher.detect(drifting)

    # 4. 検出結果の描画
    drifting = detection_renderer.draw(drifting)

    # 5. 結果の書き出し
    drifting = video_sink.write(drifting)

# 6. 終了処理
video_sink.release()


  0%|          | 0/341 [00:00<?, ?it/s]

count: 0 / 341
0: 384x640 27 persons, 942.0ms
Speed: 14.8ms preprocess, 942.0ms inference, 30.8ms postprocess per image at shape (1, 3, 384, 640)
count: 1 / 341
0: 384x640 24 persons, 778.1ms
Speed: 2.6ms preprocess, 778.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
count: 2 / 341
0: 384x640 25 persons, 777.9ms
Speed: 2.4ms preprocess, 777.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
count: 3 / 341
0: 384x640 26 persons, 791.1ms
Speed: 2.7ms preprocess, 791.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
count: 4 / 341
0: 384x640 28 persons, 796.4ms
Speed: 3.4ms preprocess, 796.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
count: 5 / 341
0: 384x640 25 persons, 802.9ms
Speed: 3.7ms preprocess, 802.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
count: 6 / 341
0: 384x640 25 persons, 806.0ms
Speed: 2.5ms preprocess, 806.0ms inference, 1.1ms postprocess per image at shape (1, 3, 